In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
import heapq
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import networkx as nx
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
stop_words = stopwords.words('english')
word_embeddings = {}
clean_sentences = []



In [ ]:
def get_data():

    #file_path = str(input("Enter path: "))
    f = open('/content/gdrive/MyDrive/project/story.txt', 'r')
    data = f.read()
    sentences = (sent_tokenize(data))
    print("Got data")
    return sentences

In [ ]:
def clean_data():

    sentences = get_data()
    sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    sentences = [s.lower() for s in sentences]

    for sen in sentences:
        sen = sen.split()
        sentence = remove_stopwords(sen)
        clean_sentences.append(sentence)

    print("cleaned sentences")

In [ ]:
def remove_stopwords(sentence):

    new_sen = ' '.join([word for word in sentence if word not in stop_words])
    return new_sen

In [ ]:
def vectorization():

    clean_data()
    f = open('/content/gdrive/MyDrive/project/story.txt', encoding = 'utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype = 'float32')
        word_embeddings[word] = coefs

    f.close()
    print('Glove job done')

In [ ]:
sentence_vectors = []
    

In [ ]:
for sentence in clean_sentences:

        if len(sentence) != 0:
             v = sum([word_embeddings.get(w, np.zeros((100,))) for w in sentence.split()])/(len(sentence.split())+0.001)
        else:
             v = np.zeros((100,))
        sentence_vectors.append(v)

In [ ]:
def similarity_matrix():

    sentence_vectors = vectorization()
    matrix = np.zeros([len(clean_sentences), len(clean_sentences)])

    for i in range(len(clean_sentences)):
        for j in range(len(clean_sentences)):
            matrix[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

    print("Similarity matrix constructed")
    return matrix

In [ ]:
def scoring_ranking(num):

    sim_matrix = similarity_matrix()
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(clean_sentences)), reverse=True)

    for i in range(num):
        print(ranked_sentences[i][1])

In [ ]:
def extract_sentences():

    number_of_sentences = int(input("Enter the number of sentences required: "))
    scoring_ranking(number_of_sentences)

extract_sentences()

Enter the number of sentences required: 20
Got data
cleaned sentences


<ipython-input-15-6594b10be895>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


IndexError: ignored

In [ ]:
import re
import string
import numpy as np
import matplotlib.pyplot as plt
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus.reader.wordnet import NOUN, VERB, ADJ, ADV
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
from operator import itemgetter
%matplotlib

Using matplotlib backend: agg


In [ ]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [ ]:
sentences = brown.sents('/content/gdrive/MyDrive/project/story.txt')

LookupError: ignored

In [ ]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
sentences = brown.sents('/content/gdrive/MyDrive/project/story.txt')

In [ ]:
len(sentences)

138

In [ ]:
sentlist=[' '.join(sent) for sent in sentences]

In [ ]:
wholesent=""
for i in sentlist:
    wholesent+=i;
print(wholesent)

IN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER,THE ONE AS MASTER, THE OTHER AS MANMr. Phileas Fogg lived, in 1872, at No. 7, Saville Row, BurlingtonGardens, the house in which Sheridan died in 1814. He was one of themost noticeable members of the Reform Club, though he seemed always toavoid attracting attention; an enigmatical personage, about whom littlewas known, except that he was a polished man of the world. People saidthat he resembled Byron--at least that his head was Byronic; but he wasa bearded, tranquil Byron, who might live on a thousand years withoutgrowing old.Certainly an Englishman, it was more doubtful whether Phileas Fogg wasa Londoner. He was never seen on 'Change, nor at the Bank, nor in thecounting-rooms of the "City"; no ships ever came into London docks ofwhich he was the owner; he had no public employment; he had never beenentered at any of the Inns of Court, either at the Temple, or Lincoln'sInn, or Gray's Inn; nor had his voice ever resounded in the Co

In [ ]:
stop_words = set(stopwords.words("english"))
punctuations = set(string.punctuation)
pos_tags = {
            NOUN: ['NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRP$', 'WP', 'WP$'],
            VERB: ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'],
            ADJ: ['JJ', 'JJR', 'JJS'],
            ADV: ['RB', 'RBR', 'RBS', 'WRB']
}

In [ ]:
def remove_stop_words(words):
        return [w for w in words if w not in stop_words]

In [ ]:
def remove_regex(sent):
        sent = " ".join([w.lower() for w in sent])
        sent = re.sub(r"i'm", "i am", sent)
        sent = re.sub(r"he's", "he is", sent)
        sent = re.sub(r"can't", "cannot", sent)
        sent = re.sub(r"don't", "do not", sent)
        sent = re.sub(r"that's", "that is", sent)
        sent = re.sub(r"\'ve", " have", sent)
        sent = re.sub(r"\'ll", " will", sent)
        sent = re.sub(r"what's", "what is", sent)
        sent = re.sub(r"where's", "where is", sent)
        sent = re.sub(r"\'re", " are", sent)
        sent = re.sub(r"\'d", " would", sent)
        sent = re.sub(r"she's", "she is", sent)
        sent = re.sub(r"won't", "will not", sent)
        patterns = re.finditer("#[\w]*", sent)
        for pattern in patterns:
            sent = re.sub(pattern.group().strip(), "", sent)
        sent = "".join(ch for ch in sent if ch not in punctuations)
        return sent

In [ ]:
def posTagging(words):
        tagged_words = pos_tag(words)
        pos_words = []
        for word in tagged_words:
            flag = False
            for key, value in pos_tags.items():
                if word[1] in value:
                    pos_words.append((word[0], key))
                    flag = True
                    break
            if not flag:
                pos_words.append((word[0], NOUN))
        return pos_words

In [ ]:
def preprocessData(sentence):
    sentence= remove_regex(sentence)
    words = word_tokenize(sentence)
    cleanedWords = remove_stop_words(words)
    lem = WordNetLemmatizer()
    pos_words = posTagging(words)
    cleanedWords = [lem.lemmatize(w, pos=p) for w, p in pos_words]
    return cleanedWords

In [ ]:
def findSentenceSimilarity(s1, s2):
    s1 = preprocessData(s1)
    s2 = preprocessData(s2)
    allWords = list(set(s1 + s2))
    vectorForS1 = [0] * len(allWords)
    vectorForS2 = [0] * len(allWords)
    for word in s1:
        vectorForS1[allWords.index(word)] += 1
    for word in s2:
        vectorForS2[allWords.index(word)] += 1
    return 1 - cosine_distance(vectorForS1, vectorForS2)

In [ ]:
def createSimilarityMatrix(sentences):
    matrix = np.zeros((len(sentences), len(sentences)))
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue
            else:
                matrix[i][j] = findSentenceSimilarity(sentences[i], sentences[j])
    for i in range(len(matrix)):
        matrix[i] /= matrix[i].sum()
    return matrix

In [ ]:
SimilarityMatrix = createSimilarityMatrix(sentences)

LookupError: ignored

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
SimilarityMatrix = createSimilarityMatrix(sentences)

LookupError: ignored

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
SimilarityMatrix = createSimilarityMatrix(sentences)

LookupError: ignored

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
SimilarityMatrix = createSimilarityMatrix(sentences)

In [ ]:
def pagerank(matrix, eps=1.0e-8, d=0.85):
    N = matrix.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    last_v = np.ones((N, 1), dtype=np.float32) * np.inf
    M_hat = (d * matrix) + (((1 - d) / N) * np.ones((N, N), dtype=np.float32))
    
    while np.linalg.norm(v - last_v, 2) > eps:
        last_v = v
        v = np.matmul(M_hat, v)
    return v

In [ ]:
ranks = pagerank(SimilarityMatrix)

In [ ]:
ranks

array([[1.29435642e-02],
       [3.42805607e-03],
       [7.49438175e-03],
       [6.56768803e-03],
       [1.33432715e-02],
       [1.27677243e-02],
       [3.15562146e-03],
       [2.48562087e-03],
       [1.05681255e-02],
       [8.82039418e-03],
       [1.05425900e-02],
       [1.15978593e-02],
       [1.34855461e-02],
       [1.05795987e-02],
       [7.28000259e-03],
       [1.88703544e-03],
       [1.21011207e-02],
       [2.59411240e-03],
       [1.39973393e-02],
       [5.05345154e-03],
       [1.81292440e-04],
       [6.54775998e-03],
       [4.67164057e-04],
       [2.13660260e-03],
       [1.32959236e-02],
       [1.07137487e-02],
       [4.07318429e-03],
       [1.39513619e-03],
       [5.75926030e-03],
       [3.88888709e-03],
       [1.40153215e-02],
       [1.12853899e-02],
       [1.16255248e-02],
       [1.38732181e-02],
       [2.29035702e-03],
       [1.36122694e-04],
       [1.02971112e-04],
       [9.94675900e-03],
       [6.21499425e-03],
       [1.20245820e-02],


In [ ]:
sortRankWithindexes = [item[0] for item in sorted(enumerate(ranks), key=lambda item: -item[1])]

In [ ]:
sortRankWithindexes

[30,
 18,
 33,
 61,
 113,
 94,
 88,
 116,
 12,
 4,
 24,
 99,
 0,
 136,
 87,
 63,
 5,
 46,
 42,
 96,
 79,
 65,
 16,
 39,
 82,
 32,
 11,
 77,
 67,
 75,
 102,
 31,
 115,
 137,
 74,
 25,
 114,
 51,
 13,
 8,
 10,
 123,
 120,
 47,
 83,
 37,
 80,
 90,
 89,
 106,
 58,
 54,
 125,
 56,
 118,
 119,
 9,
 81,
 128,
 126,
 103,
 60,
 108,
 131,
 64,
 111,
 2,
 93,
 73,
 14,
 130,
 98,
 117,
 104,
 3,
 21,
 122,
 38,
 52,
 135,
 91,
 84,
 95,
 68,
 28,
 100,
 44,
 66,
 57,
 105,
 45,
 19,
 121,
 40,
 129,
 132,
 26,
 86,
 101,
 29,
 62,
 110,
 50,
 134,
 69,
 112,
 1,
 109,
 107,
 59,
 133,
 53,
 6,
 97,
 49,
 17,
 43,
 7,
 34,
 23,
 15,
 85,
 78,
 72,
 27,
 48,
 92,
 71,
 55,
 22,
 124,
 127,
 76,
 70,
 20,
 35,
 36,
 41]

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar([item[0] for item in sorted(enumerate(ranks))], ranks.T[0],color='olive',width=0.8)
plt.ylabel("Page Rank / Importance")
plt.xlabel("Sentence No.")
plt.show()

In [ ]:
SummaryLines = 5

In [ ]:
selectedSentences = sorted(sortRankWithindexes[:SummaryLines])

In [ ]:
summary = itemgetter(*selectedSentences)(sentences)

In [ ]:
finalSummary=""
for sentence in summary:
    finalSummary+=' '.join(sentence)

In [ ]:
finalSummary

'a merchant or a gentleman farmer. His name was strange to theHis cheques were regularly paid at sight from his account current,not imagine how he had made his fortune, and Mr. Fogg was the lastPhileas Fogg was not known to have either wife or children, which maygymnastics, so as to make better use of my talents; and then I was a'

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Pagerank2.txt')
n  = (doc1.similarity(doc2))
print (n*100)

97.6822554014052


<ipython-input-64-88925d49c1c4>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum2.0.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Pagerank2.txt')
n  = (doc1.similarity(doc2))
print (n*100)

99.07579614544817


<ipython-input-65-6bbbf08852b8>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum3.0.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Pagerank2.txt')
n  = (doc1.similarity(doc2))
print (n*100)

96.41563318472308


<ipython-input-66-2efab4dac04e>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))
